In [11]:
import pandas as pd

In [ ]:
df = pd.read_json('latest_grocery_data.json')

In [ ]:
df.columns

In [ ]:
df = df.dropna()

In [ ]:
df.drop_duplicates()

In [ ]:
# Split units into quantity + unit
units_split = df['units'].str.extract(r'(?P<unit_quantity>\d+(?:\.\d+)?)\s*(?P<unit>[a-zA-Z]+)')
df = df.assign(**units_split)
df[['units', 'unit_quantity', 'unit']].head()

In [ ]:
df

In [ ]:
df = df.drop(
    columns=[
        "concatted",
        "old_price",
        "price_per_unit",
        "other",
        "unit",
    ],
)

In [ ]:
df

In [ ]:
import re

# Heuristic aisle assignment based on product_name keywords
AISLE_RULES = [
    ('Produce', [
        r'\bapple(s)?\b', r'\bbanana(s)?\b', r'\bavocado(s)?\b', r'\bberry(ies)?\b',
        r'\bgrape(s)?\b', r'\bcitrus\b', r'\blemon(s)?\b', r'\blime(s)?\b',
        r'\borange(s)?\b', r'\btomato(es)?\b', r'\bonion(s)?\b', r'\bgarlic\b',
        r'\blettuce\b', r'\bspinach\b', r'\bcarrot(s)?\b', r'\bcucumber(s)?\b',
        r'\bpepper(s)?\b', r'\bbroccoli\b', r'\bcauliflower\b', r'\bpotato(es)?\b',
        r'\bmushroom(s)?\b', r'\bherb(s)?\b', r'\bcilantro\b', r'\bparsley\b',
        r'\bbasil\b', r'\bsalad\b', r'\bfruit\b', r'\bvegetable(s)?\b',
    ]),
    ('Dairy', [
        r'\bmilk\b', r'\bcheese\b', r'\byogurt\b', r'\bbutter\b', r'\bcream\b',
        r'\bcheddar\b', r'\bmozzarella\b', r'\bparmesan\b', r'\bcottage cheese\b',
        r'\bsour cream\b', r'\bwhipped\b', r'\bhalf\s*&\s*half\b', r'\bkefir\b',
        r'\begg(s)?\b',
    ]),
    ('Meat', [
        r'\bbeef\b', r'\bpork\b', r'\bchicken\b', r'\bturkey\b', r'\bsausage\b',
        r'\bbacon\b', r'\bham\b', r'\bsteak\b', r'\bground\b', r'\bsalami\b',
        r'\bpepperoni\b', r'\blamb\b', r'\bseafood\b', r'\bshrimp\b', r'\bfish\b',
        r'\bsalmon\b', r'\btuna\b', r'\bcod\b', r'\bcrab\b', r'\blobster\b',
        r'\bdeli\b', r'\bprosciutto\b',
    ]),
    ('Frozen', [
        r'\bfrozen\b', r'\bice cream\b', r'\bgelato\b', r'\bpopsicle(s)?\b',
        r'\bice pop(s)?\b', r'\bfrozen pizza\b', r'\bfries\b', r'\bnugget(s)?\b',
    ]),
    ('Beverages', [
        r'\bsoda\b', r'\bjuice\b', r'\bwater\b', r'\btea\b', r'\bcoffee\b',
        r'\bbeer\b', r'\bwine\b', r'\bcider\b', r'\bkombucha\b', r'\bsparkling\b',
        r'\bdrink(s)?\b', r'\bcola\b', r'\benergy\b', r'\bsports drink\b',
    ]),
    ('Bakery', [
        r'\bbread\b', r'\bbagel(s)?\b', r'\bbun(s)?\b', r'\broll(s)?\b',
        r'\bcroissant(s)?\b', r'\bmuffin(s)?\b', r'\bcake\b', r'\bcookie(s)?\b',
        r'\bpastry\b', r'\bdonut(s)?\b', r'\btortilla(s)?\b', r'\bpita\b', r'\bnaan\b',
    ]),
    ('Snacks', [
        r'\bchip(s)?\b', r'\bcracker(s)?\b', r'\bpopcorn\b', r'\bpretzel(s)?\b',
        r'\btrail mix\b', r'\bgranola bar(s)?\b', r'\bprotein bar(s)?\b',
        r'\bsnack(s)?\b', r'\bnut(s)?\b', r'\bcandy\b', r'\bchocolate\b',
    ]),
    ('Pantry', [
        r'\bpasta\b', r'\brice\b', r'\bflour\b', r'\bsugar\b', r'\bsalt\b',
        r'\bspice(s)?\b', r'\boil\b', r'\bvinegar\b', r'\bsauce\b', r'\bketchup\b',
        r'\bmustard\b', r'\bmayonnaise\b', r'\bcereal\b', r'\boats\b', r'\bbean(s)?\b',
        r'\blentil(s)?\b', r'\bsoup\b', r'\bbroth\b', r'\bcanned\b', r'\bjar\b',
        r'\bcan\b', r'\bpackaged\b',
    ]),
]

def assign_aisle(name):
    if not isinstance(name, str):
        return 'Other'
    text = name.lower()
    for aisle, patterns in AISLE_RULES:
        for pattern in patterns:
            if re.search(pattern, text):
                return aisle
    return 'Other'

df['aisle'] = df['product_name'].apply(assign_aisle)
df['aisle'].value_counts().head()


In [ ]:
df.to_json('cleaned_grocery_data.json', orient='records', lines=True)